---
title: "FYS3220 -- LAB B: Common electrical circuits"
author: "John Isaac Calderon"
exports:
  - format: pdf
    template: plain_latex
    article_type: Report
numbering:
    heading_1: true
    heading_2: true
    heading_3: true
---

# Administration - preamble
If you are seeing this section, then this notebook has been improperly rendered - and will be littered with code and other cells that are supposed to be hidden from general readers.

In [1]:
import os
import sys
import shutil

print(" I: Executing preamble code...", file=sys.stderr)

 I: Executing preamble code...


In [2]:
# --- Preamble code START --- #

from IPython.display import display, Markdown, Latex
import matplotlib.pyplot as plt
import numpy as np
import math

# - Definitions - #

# Set the notebook name
docname = "FYS3220_LAB_B_V25_jicalder"
nbname = f"{docname}.ipynb"
pdfname = f"{docname}.pdf"

cwd = os.path.realpath(".")
assert len(cwd) > 0 and cwd != "/"

# Initialize temporary figures directory
figsdir = "tmpfigs"
figsdir_full = f"{cwd}/{figsdir}"

if os.path.exists(figsdir):
    shutil.rmtree(figsdir)

os.mkdir(figsdir)

print(f"  > Current directory: {cwd}", file=sys.stderr)
print(f"  > Target notebook: {nbname}", file=sys.stderr)
print(f"  > Output file: {pdfname}", file=sys.stderr)
print(f"  > Figures directory: {figsdir_full}", file=sys.stderr)

# Define a counter variable for the figures
FIG_CTR = 1


# - Functions - #

# Clear the page
def clearpage() -> None:
    display(Latex(r"\clearpage"))
    
# Display a generic image figure
def disp_imgfig(img: str, desc: str, preamble: str = "") -> None:
    global FIG_CTR

    # Make sure the file even exists
    if not os.path.exists(img):
        raise ValueError(f"Provided image file '{img}' does not exist.")

    # Create caption
    caption = f"**Figure {FIG_CTR}:** {desc}"

    # Create Markdown sequence
    md = f"|![{desc}]({img})|\n|:--:|\n|{caption}|"

    if preamble:
        md = preamble + "\n\n" + md

    # Display the sequence
    display(Markdown(md))

    # Increment the figure counter
    FIG_CTR += 1


# Display two generic image figures
def disp_imgfig2(img1: str, img2: str, desc1: str, desc2: str, preamble: str = "") -> None:
    global FIG_CTR

    # Make sure the files even exist
    if not os.path.exists(img1):
        raise ValueError(f"Provided image file '{img1}' does not exist.")

    if not os.path.exists(img2):
        raise ValueError(f"Provided image file '{img2}' does not exist.")

    # Create caption
    caption1 = f"**Figure {FIG_CTR}:** {desc1}"
    caption2 = f"**Figure {FIG_CTR+1}:** {desc2}"

    caption = caption1 + "\\\n" + caption2
    
    # Create Markdown sequence
    md = f"|![{desc1}]({img1})|![{desc2}]({img2})|\n|:--:|:--:|\n|{caption1}|{caption2}|"

    if preamble:
        md = preamble + "\n\n" + md

    # Display the sequence
    display(Markdown(md))

    # Increment the figure counter
    FIG_CTR += 2


# Display a Pyplot figure
def disp_pltfig(fig: plt.Figure, desc: str, preamble: str = "") -> None:    
    global FIG_CTR
    
    # Save the figure as an image file, using the current
    # figure counter to discriminate between figures
    figfile = f"{figsdir}/tmpfig_{FIG_CTR}.png"

    # Save the figure and close it, so that
    # the notebook isn't littered with
    # orphaned figures
    fig.savefig(figfile, bbox_inches="tight", dpi=400)
    plt.close(fig)

    disp_imgfig(figfile, desc, preamble)


# Display two Pyplot figures
def disp_pltfig2(fig1: plt.Figure, fig2: plt.Figure, desc1: str, desc2: str, preamble: str = "") -> None:    
    global FIG_CTR
    
    # Save the figure as an image file, using the current
    # figure counter to discriminate between figures
    figfile_1 = f"{figsdir}/tmpfig_{FIG_CTR}.png"
    figfile_2 = f"{figsdir}/tmpfig_{FIG_CTR+1}.png"

    # Save the figure and close it, so that
    # the notebook isn't littered with
    # orphaned figures
    fig1.savefig(figfile_1, bbox_inches="tight", dpi=400)
    fig2.savefig(figfile_2, bbox_inches="tight", dpi=400)
    plt.close(fig1)
    plt.close(fig2)

    disp_imgfig2(figfile_1, figfile_2, desc1, desc2, preamble)

# Find the `x` coordinates that satisfy `y1[...] == y2`
def where_is(
    x: np.ndarray, 
    y1: np.ndarray, 
    y2: float, 
    err: float = 1.0e-3
) -> list[float]:
    assert len(x) == len(y1)

    a = []

    for u, v in zip(x, y1):
        if math.isclose(v, y2, rel_tol=err):
            a.append(u)

    return np.array(a)
# --- Preamble code END --- #

  > Current directory: /home/johnc/Documents/Notebooks/FYS3220_LAB_B_V25_jicalder
  > Target notebook: FYS3220_LAB_B_V25_jicalder.ipynb
  > Output file: FYS3220_LAB_B_V25_jicalder.pdf
  > Figures directory: /home/johnc/Documents/Notebooks/FYS3220_LAB_B_V25_jicalder/tmpfigs


In [3]:
print(" I: Finished executing preamble code", file=sys.stderr)

 I: Finished executing preamble code


There really isn't much to do, other than to pull through the tasks at hand...

In [4]:
# Mostly following the guide made by UC Berkeley
# (origin: https://pythonnumericalmethods.studentorg.berkeley.edu/notebooks/chapter24.04-FFT-in-Python.html)
from numpy.fft import fft, ifft
from scipy.signal import find_peaks
from typing import Optional

# Define the normalized FFT function
# ...which is really just a shorthand for
# "do everything we would otherwise have
# to write down every time"
# - I find the function signature incredibly funny
# for some reason...
# - The times in `t` should be equidistant
# - the coefficient it returns are complex, because...
def norm_fft(
    t: np.ndarray, 
    x: np.ndarray,
    thr: Optional[float] = None,
) -> (np.ndarray, np.ndarray, np.ndarray):
    # Calculate the sampling rate
    dt = t[1] - t[0]
    f_s = 1 / dt

    # Perform FFT on the input
    X = fft(x)            # FFt of `x`
    N = len(X)            # Number of discrete frequencies
    
    assert N != 0
    
    n = np.arange(N)      # Bin number
    bins = n / N * f_s    # Frequency bins
    X *= 2/N              # Normalize `X` to obtain coefficients
                          # - accounting for symmetry
    
    # - exploit the symmetry of FFT
    bins = bins[:N//2]
    X = X[:N//2]

    # Find peaks
    X_mag = np.abs(X)     # Magnitude of `X`
    peaks, _ = find_peaks(X_mag, threshold=thr)    # Bins with peaks

    return bins, X, peaks

In [5]:
# Return an array of sinusoids with common time and different parameters
def sinusoid(t: np.ndarray, f: np.ndarray, b: np.ndarray) -> np.ndarray:
    # Copy the arrays, which are otherwise mutable
    f_v, t_v, b_v = np.copy(f), np.copy(t), np.copy(b)

    # Turn the arrays into vectors with appropriate shapes
    f_v.shape = (len(f),1)
    t_v.shape = (1,len(t))
    b_v.shape = (1,len(b))

    # Calculate the argument angles, then calculate the result
    phi = 2 * np.pi * (f_v * t_v)
    res = np.sum(b_v.T * np.sin(phi), axis=0)
    return res

# RL circuit and time constant
## Schematic

In [6]:
disp_imgfig("1_RL_circuit.png", "A simple RL circuit with a manual switch")

|![A simple RL circuit with a manual switch](1_RL_circuit.png)|
|:--:|
|**Figure 1:** A simple RL circuit with a manual switch|

## Symbolic analysis
From Kirchhoff's voltage law, we have the sum of the voltages throughout the circuit:
$$\sum{U} = -V + V_R + V_L = 0$$

We then have $V_R = iR$ from Ohm's law and
$$V_L = L\frac{\mathrm{d}i}{\mathrm{d}t}$$
from Faraday's law. Henceforth
$$-V + iR + L\frac{\mathrm{d}i}{\mathrm{d}t} = 0$$
which is a first-order non-homogeneous ordinary differential equation. We can rewrite the equation as
$$\frac{\mathrm{d}i}{\mathrm{d}t} + \frac{R}{L}\left(i - \frac{V}{R}\right) = 0$$
which also makes the system's principle of function easier to read at a glance. Using the substitution $I = i - V/R$ and the fact that $\dot{V} = 0$, we get the
homogeneous ODE
$$\frac{\mathrm{d}I}{\mathrm{d}t} + \frac{R}{L}I = 0$$
which has solutions of the form
$$I(t) = I(0) \cdot e^{-(R/L)t}$$
with $I(0) = i(0) - V/R$ being the initial value for $I$. Reversing the substitution yields
$$i(t) - \frac{V}{R} = \left[i(0) - \frac{V}{R}\right]e^{-(R/L)t}$$
and with $i(0) = 0$, we get
$$i(t) = \left[1-e^{-(R/L)t}\right]\frac{V}{R}$$
Finally, the time constant of the system is $\tau = L/R$, so that $t/\tau = (R/L)t$. The time constant is essentially the time it takes for the system to reach (or lose) $1-1/e \approx 63.2\%$ of its value.
ts $T = 5\tau$ would yield $1-1/e^5 \approx 99.3\%$.


In [7]:
clearpage()

<IPython.core.display.Latex object>


### Theoretical calculations
Using the result $\tau = L/R$ and $L = 100\,\mathrm{mH}$, the resistance that yields $\tau = 1\,\mathrm{ms}$ is
$$R = \frac{100\,\mathrm{mH}}{1\,\mathrm{ms}} \approx 100\,\Omega$$

The number of time constants it would require for the system to reach 99% of its maximum value can be calculated as follows:
$$T_{99\%}/\tau \geq \ln\left(\frac{1}{1-0.99}\right) \approx 4.605$$
A time of five time constants should put the system at 99.3% of its maximum value.

## Results

In [8]:
disp_imgfig("result_task_1-1.png", "Transient analysis of the RL circuit")

|![Transient analysis of the RL circuit](result_task_1-1.png)|
|:--:|
|**Figure 2:** Transient analysis of the RL circuit|

As can be seen in figure 2, from the time the switch closes, to the time the current reaches 99% of its maximum value, $T = 5.0\,\mathrm{ms}$ has elapsed -- and with a time constant of $\tau = 1\,\mathrm{ms}$, this just confirms that the time it takes for the current to reach 99% of its maximum value is 5 time constants.

In [9]:
display(Latex(r"\clearpage"))

<IPython.core.display.Latex object>

# RLC circuit: transient analysis
## Schematic

In [10]:
disp_imgfig("lab-b-task-1_2-schematic.png", "A collection of series RLC branches in a single circuit")

|![A collection of series RLC branches in a single circuit](lab-b-task-1_2-schematic.png)|
|:--:|
|**Figure 3:** A collection of series RLC branches in a single circuit|

## Symbolic analysis
We are already given the transadmittance
$$
Y(s) = \frac{\frac{1}{L}s}{s^2 + \frac{R}{L}s + \frac{1}{LC}}
$$
and the the equation for the poles
$$
    s^2 + \frac{R}{L}s + \frac{1}{LC} = 0
$$
with the solutions
$$
p_1,p_2 = -\alpha \pm \beta = -\frac{R}{2L} \pm \sqrt{\left(-\frac{R}{2L}\right)^2 - \frac{1}{LC}}
$$
This ultimately leads to
\begin{align*}
    \alpha = \frac{R}{2L} && \beta = \sqrt{\left(\frac{R}{2L}\right)^2 - \frac{1}{LC}}
\end{align*}
We can see that the numerator in $Y(s)$ has the trivial zero $s = 0$, which makes sense, since a DC signal would be blocked by the series capacitance. This ultimately means that we only need to focus on the poles.

### Numerical calculations
The only thing we need to do, is to calculate $\alpha$ and $\beta$, then plug them in into the table below:

In [11]:
_md = rf"""
|    |Poles  |$\alpha$ [kHz] |$\beta$ [kHz] |$p_1$ [kHz] |$p_2$ [kHz] |
|:--:|:-----:|:-------------:|:------------:|:----------:|:----------:|
"""

R = [2000.0, 200.0, 20.0]  # branch resistance
L = 10.0e-3  # branch inductance
C = 1.0e-6  # branch capacitance

for n in range(1,4):
    alpha = R[n-1] / (2 * L) / 1000
    beta = pow(pow(alpha*1000, 2) - 1/(L * C), 1/2) / 1000
    N = 1 if beta == 0.0 else 2
    _md += f"|$R_{n}$ branch|${N}$|${alpha:.1f}$|${beta:.1f}$|${-alpha+beta:.1f}$|${-alpha-beta:.1f}$|\n"
display(Markdown(_md))


|    |Poles  |$\alpha$ [kHz] |$\beta$ [kHz] |$p_1$ [kHz] |$p_2$ [kHz] |
|:--:|:-----:|:-------------:|:------------:|:----------:|:----------:|
|$R_1$ branch|$2$|$100.0$|$99.5$|$-0.5$|$-199.5$|
|$R_2$ branch|$1$|$10.0$|$0.0$|$-10.0$|$-10.0$|
|$R_3$ branch|$2$|$1.0$|$0.0+9.9j$|$-1.0+9.9j$|$-1.0-9.9j$|


While the values above are numerically correct, they can be highly misleading, since it isn't immediately obvious that the values represent *angular frequencies*, and not *temporal frequencies* -- the latter of which are, so to speak, "human-readable".

In [12]:
_md = rf"""
|    |$\alpha/(2\pi)$ [kHz] |$\beta/(2\pi)$ [kHz] |$p_1/(2\pi)$ [Hz] |$p_2/(2\pi)$ [Hz] |
|:--:|:--------------------:|:-------------------:|:----------------:|:----------------:|
"""

import math

R = [2000.0, 200.0, 20.0]  # branch resistance
L = 10.0e-3  # branch inductance
C = 1.0e-6  # branch capacitance

for n in range(1,4):
    alpha = R[n-1] / (2 * L)
    beta = pow(pow(alpha, 2) - 1/(L * C), 1/2)

    alpha_h = alpha / (2 * math.pi)
    beta_h = beta / (2 * math.pi)
    
    N = 1 if beta == 0.0 else 2
    _md += f"|$R_{n}$ branch|${alpha_h/1000:.2f}$|${beta_h/1000:.2f}$|${-alpha_h+beta_h:.0f}$|${-alpha_h-beta_h:.0f}$|\n"
display(Markdown(_md))


|    |$\alpha/(2\pi)$ [kHz] |$\beta/(2\pi)$ [kHz] |$p_1/(2\pi)$ [Hz] |$p_2/(2\pi)$ [Hz] |
|:--:|:--------------------:|:-------------------:|:----------------:|:----------------:|
|$R_1$ branch|$15.92$|$15.84$|$-80$|$-31751$|
|$R_2$ branch|$1.59$|$0.00$|$-1592$|$-1592$|
|$R_3$ branch|$0.16$|$0.00+1.58j$|$-159+1584j$|$-159-1584j$|


## Results
### Transient analysis

In [13]:
disp_imgfig("result_task_1-2-a.png", 
            "Transient analysis of the currents through the RLC branches. \
            The currents are appropriately scaled for sufficient separation.")

|![Transient analysis of the currents through the RLC branches.             The currents are appropriately scaled for sufficient separation.](result_task_1-2-a.png)|
|:--:|
|**Figure 4:** Transient analysis of the currents through the RLC branches.             The currents are appropriately scaled for sufficient separation.|

In [14]:
clearpage()

<IPython.core.display.Latex object>

### AC analysis

In [15]:
disp_imgfig("result_task_1-2-b.png", "A Bode plot of the currents with respect to *temporal frequency*")

|![A Bode plot of the currents with respect to *temporal frequency*](result_task_1-2-b.png)|
|:--:|
|**Figure 5:** A Bode plot of the currents with respect to *temporal frequency*|

## Discussion
### Transient analysis
There really isn't much to discuss, other than the fact that
 - the $R_1$ branch (figure 4; orange trace) exhibits overdamped oscillation
 - the $R_2$ branch (figure 4; yellow trace) exhibits critically damped oscillation
 - the $R_3$ branch

### AC analysis
We can observe that, for real $\beta \neq 0$, we get two real poles -- which we can also observe by looking where the magnitude is exactly -3 decibels below the maximum magnitude. This is the case for the $R_1$ branch (see figure 5; blue trace), which exhibits an overdamped oscillation (see figure 4; orange trace).

We then have the case with $\beta = 0$, meaning that the transadmittance immediately transitions from *increasing* to *decreasing* past the system's sole pole. This is the case for the $R_2$ branch (see figure 5; orange trace), which exhibits a critically damped oscillation (see figure 4; yellow trace).

Finally, we have the case with imaginary $\beta$, where we get two complex poles -- but only one peak in the frequency domain. It can be observed that the frequencies where the magnitude is -3 decibels below the maximum magnitude, is
$$\omega_{-3\,\mathrm{dB}} = \alpha \mp j\beta$$
where $\omega_{-3\,\mathrm{dB}}$ are real frequencies. This is very much the case for the $R_3$ branch (see figure 5; yellow trace), which exhibits an underdamped oscillation (see figure 4; blue trace).

Our observations establish the connection between the Bode plots and the table in this section.

# Ideal and non-ideal operational amplifiers
## Schematics

### Individual amplifiers

In [16]:
disp_imgfig("lab-b-task-2_1-schematic.png", "Four inverting amplifiers with different gains")

|![Four inverting amplifiers with different gains](lab-b-task-2_1-schematic.png)|
|:--:|
|**Figure 6:** Four inverting amplifiers with different gains|

In [17]:
clearpage()

<IPython.core.display.Latex object>

## Chained amplifiers

In [18]:
disp_imgfig("lab-b-task-2_2-schematic.png", "Two inverting amplifiers connected in series.")

|![Two inverting amplifiers connected in series.](lab-b-task-2_2-schematic.png)|
|:--:|
|**Figure 7:** Two inverting amplifiers connected in series.|

## Symbolic analysis
Assuming that the amplifiers in figure 6 are ideal (realistically; that we are operating in their near-ideal range), using the fact that amplifiers will always "attempt" to "equalize" the voltages on its two inputs, we obtain the following relation:
$$
\left(V_1 - V_\mathrm{out}\right) \frac{R_2}{R_1 + R_2} = -V_\mathrm{out} \Longleftrightarrow \left(V_1 - V_\mathrm{out}\right)R_2 = -\left(R_1 + R_2\right)V_\mathrm{out}
$$
Simplifying the statement on the right-hand side, we get $V_1 R_2 = -R_1 V_\mathrm{out}$, or equivalently
$$
H = \frac{V_\mathrm{out}}{V_1} = -\frac{R_2}{R_1}
$$
which is the result we are looking for.

In [19]:
clearpage()

<IPython.core.display.Latex object>

### Numerical calculations

In [20]:
_md = rf"""
|    |Calculated gain |Logarithmic gain [dB] |
|:--:|:--------------:|:--------------------:|
"""

import math

R1 = 10.0e3   # input resistance
R2 = [10.0e6, 1.0e6, 100.0e3, 10.0e3]    # feedback resistance

for n in range(1,4):
    gain = -R2[n-1] / R1
    log_gain = 20 * math.log10(abs(gain))
    
    _md += f"|Amplifier {n} |${gain:.1f}$ |${log_gain:.1f}$ |\n"
display(Markdown(_md))


|    |Calculated gain |Logarithmic gain [dB] |
|:--:|:--------------:|:--------------------:|
|Amplifier 1 |$-1000.0$ |$60.0$ |
|Amplifier 2 |$-100.0$ |$40.0$ |
|Amplifier 3 |$-10.0$ |$20.0$ |


## Results
### Individual amplifiers

In [21]:
disp_imgfig("result_task_2-1.png", 
            "Bode plots for the four inverting amplifiers. \
            The frequency where the output drops by -3 decibels, is the amplifier's bandwidth")

|![Bode plots for the four inverting amplifiers.             The frequency where the output drops by -3 decibels, is the amplifier's bandwidth](result_task_2-1.png)|
|:--:|
|**Figure 8:** Bode plots for the four inverting amplifiers.             The frequency where the output drops by -3 decibels, is the amplifier's bandwidth|

We have the Bode plots for the four amplifers above, with the -3 dB frequencies listed in the table below.

In [22]:
file = open("result_task_2-1_bode.csv")   # open file
n = sum(len(i) != 0 for i in file.readline().strip().split(",")[1:])   # find the number of non-empty columns, except frequency

f = []   # frequency
x = []   # output values

for l in file.readlines():
    r = l.strip().split(",")
    try:
        freq = float(eval(r[0])) 
        v = [float(eval(r[1+i])) for i in range(n)]
    except:
        continue
    f.append(freq)
    x.append(v)

f_a = np.array(f)
x_a = np.array(x)

index_low = lambda i: np.max(np.where(x_a[:, i] >= (np.max(x_a[:, i]) - 3.0102)))
index_high = lambda i: np.min(np.where(x_a[:, i] <= (np.max(x_a[:, i]) - 3.0102)))

x_0 = np.power(10, x_a[0, :]/20)

f_low = np.array([f_a[index_low(i)] for i in range(n)])   # lower bound -3 dB frequencies
f_high = np.array([f_a[index_high(i)] for i in range(n)])  # upper bound -3 dB frequencies
f_th = (f_low + f_high) / 2   # middle -3 dB frequencies

_md = rf"""
|    |Gain $A$ [1] |Maximum frequency $f_{{-3\,\mathrm{{dB}}}}$ [Hz] |GBW [Hz] |
|:--:|:-----------:|:-----------------------------------------------:|:-------:|
"""

for i in range(n):
    _md += f"|Amplifier {i+1} |${x_0[i]:.1f}$|${f_th[i]:.0f}$|${x_0[i] * f_th[i]:.1f}$|\n"
display(Markdown(_md))


|    |Gain $A$ [1] |Maximum frequency $f_{-3\,\mathrm{dB}}$ [Hz] |GBW [Hz] |
|:--:|:-----------:|:-----------------------------------------------:|:-------:|
|Amplifier 1 |$999.0$|$989$|$987629.5$|
|Amplifier 2 |$100.0$|$9886$|$988518.8$|
|Amplifier 3 |$10.0$|$90163$|$901621.0$|
|Amplifier 4 |$1.0$|$495483$|$495482.0$|


In [23]:
clearpage()

<IPython.core.display.Latex object>

### Chained amplifiers

In [24]:
disp_imgfig("result_task_2-2.png", 
            "Bode plots for the two inverting amplifiers in question. \
            The frequency where the output drops by -3 decibels, is the amplifier's bandwidth")

|![Bode plots for the two inverting amplifiers in question.             The frequency where the output drops by -3 decibels, is the amplifier's bandwidth](result_task_2-2.png)|
|:--:|
|**Figure 9:** Bode plots for the two inverting amplifiers in question.             The frequency where the output drops by -3 decibels, is the amplifier's bandwidth|

## Discussion
An idealized operational has theoretically infinite input impedance, zero output impedance and infinite open-loop gain -- whereas a real-world operational amplifier will have finite input impedance, non-zero output impedance and finite open-loop gain. 

In addition, the closed-loop gain of an idealized amplifier will -- depending on the model being used -- either remain constant for all frequencies, or strictly obey the gain-bandwidth product relation: $\mathrm{GBW} = A\,(\mathrm{gain}) \cdot f_\mathrm{BW}\,(\mathrm{bandwidth})$. Real-world amplifiers exhibit some form of gain-bandwidth relation, but will -- under specific conditions -- deviate from the proportional relation.

### Individual amplifiers
We can observe in figure 7 that the amplifiers behave like ideal amplifiers for *very low* frequencies. As the frequency increases, the gain starts decreasing until the trend settles at -20 decibels per decade.

|   |Output 1 |Output 2 |Output 3 |Output 4 |
|:-:|:-------:|:-------:|:-------:|:-------:|
|Comparable gains at $f = 100\,\mathrm{Hz}$? |Yes |Yes |Yes |Yes |
|Comparable gains at $f = 100\,\mathrm{kHz}$? |No |No |No (-3 dB) |Yes |
|Closed-loop bandwidth [kHz] |$0.989$ |$9.886$ |$90.163$ |$495.483$ |

We can see in the results subsection that, for $A > 1$ and $f < 1.0 \,\mathrm{GHz}$, the gain-bandwidth relation is relatively tame. For $f > 1.0\,\mathrm{GHz}$, the gain-bandwidth relation starts to break down.

In [25]:
clearpage()

<IPython.core.display.Latex object>

### Chained amplifiers
It doesn't seem as though the performance characteristics of the chained amplifiers has significantly improved compared to that of a single amplifier -- in fact, the bandwidth for the chained amplifiers has slightly decreased.

|   |Output 3 |New output 4 |
|:-:|:-------:|:-----------:|
|Gain [dB] |$10.0$ |$10.0$ |
|Bandwidth [kHz] |$239.5$ |$200.2$ |

# Differentiation
## Schematics

In [26]:
disp_imgfig("lab-b-task-3_1-schematic.png", "Circuit diagram of a basic non-inverting differentiator")

|![Circuit diagram of a basic non-inverting differentiator](lab-b-task-3_1-schematic.png)|
|:--:|
|**Figure 10:** Circuit diagram of a basic non-inverting differentiator|

## Symbolic analysis
Assuming that the amplifers are ideal, the relation between the non-inverting and the inverting inputs can be expressed as
$$
    \left(V_1 - V_\mathrm{mid}\right) \frac{j\omega L}{R_1 + j\omega L} = -V_\mathrm{mid} \Longleftrightarrow \left(V_1 - V_\mathrm{mid}\right)j\omega L = -\left(R_1 + j\omega L\right)V_\mathrm{mid}
$$
which can be simplified to
$$
    V_1j\omega L = -R_1 V_\mathrm{mid} \Longleftrightarrow V_\mathrm{mid} = -\frac{L}{R_1}\left(j\omega V_1\right)
$$
Applying the inverse Fourier transform on both sides the equality yields
$$
    v_\mathrm{mid}(t) = -\frac{L}{R_1}\frac{\mathrm{d}v_1}{\mathrm{d}t}(t)
$$
Since our non-inverting differentiator has an inverting unity amplifier in the second stage, we ultimately get
$$
    v_\mathrm{out}(t) = \frac{L}{R_1}\frac{\mathrm{d}v_1}{\mathrm{d}t}(t)
$$

### Example with a triangle wave
Suppose we have a triangle wave of the form
$$
    v_0(t) = \left\{\begin{matrix}
        \hat{v} - (4\hat{v}/T)t, & 0 \leq t < T/2 \\
        -3\hat{v} + (4\hat{v}/T)t, & T/2 \leq t < T
    \end{matrix}\right.
$$
where $T$ is the period of the triangle wave (in seconds) and $\hat{v}$ is the amplitude of the triangle wave (in volts). We then feed $v_0$ into the differentiator, so that
$$
    v_\mathrm{out}(t) = \frac{L}{R_1}\frac{\mathrm{d}v_0}{\mathrm{d}t}(t) = \left\{\begin{matrix}
        -4\hat{v}L/(R_1 T), & 0 < t < T/2 \\
        4\hat{v}L/(R_1 T), & T/2 < t < T
    \end{matrix}\right.
$$
which is a square wave function. We observe that the voltage is scaled by some coefficient $k = 4L/(R_1 T)$. Using $T = 4.00\,\mathrm{ms}$, $L = 700\,\mathrm{mH}$ and $R_1 = 1.2\,\mathrm{k}\Omega$, we get
$$
    k = \frac{4 \cdot 700\,\mathrm{mH}}{1.20\,\mathrm{k}\Omega \cdot 4.00\,\mathrm{ms}} \approx 0.583
$$
which means that if $\hat{v} = 1.00\,\mathrm{V}$, we can expect the output voltage magnitude to be $\hat{v}_\mathrm{out} = 0.583\,\mathrm{V}$.

## Results

In [27]:
disp_imgfig("result_task_3-1.png", 
            r"Transient analysis of a non-inverting differentiator " + \
            r"($R_1 = 1.20\,\mathrm{k}\Omega$, $L = 700\,\mathrm{mH}$)")

|![Transient analysis of a non-inverting differentiator ($R_1 = 1.20\,\mathrm{k}\Omega$, $L = 700\,\mathrm{mH}$)](result_task_3-1.png)|
|:--:|
|**Figure 11:** Transient analysis of a non-inverting differentiator ($R_1 = 1.20\,\mathrm{k}\Omega$, $L = 700\,\mathrm{mH}$)|

## Discussion

### Transient analysis
An interesting quirk that we can observe in the time plot of the output voltage, is the spikes at the edges. Other than that, the analysis isn't particularly interesting -- simply because the circuit works as expected, with correct voltage levels and function behaviors.

### Sinusoidal input
We know from basic calculus that the derivative of a sinusoidal function is another sinusoidal function that is scaled by its frequency and phase-shifted by $+90^\circ$. More precisely,
\begin{align*}
    \frac{\mathrm{d}}{\mathrm{d}t}\sin(\omega t) = \omega\cos(\omega t) && \frac{\mathrm{d}}{\mathrm{d}t}\cos(\omega t) = -\omega\sin(\omega t)
\end{align*}
meaning that if we were to use $v(t) = \hat{v}\cos(\omega t)$, then we can reasonably expect the output voltage to be
$v_\mathrm{out}(t) = -k\hat{v}\sin(\omega t)$, with $k = \omega L / R$ being our scaling factor. 

Interestingly enough, the scaling factor has a physical interpretation to it, since the numerator is the reactance of the differentiator's inductor as a function of frequency. This means that the differentiator functions as a high-pass filter -- letting high-frequency signals pass through, while impeding low-frequency signals.

# Integration
## Schematics

In [28]:
disp_imgfig("lab-b-task-4_1-schematic.png", "Circuit diagram of a basic non-inverting integrator")

|![Circuit diagram of a basic non-inverting integrator](lab-b-task-4_1-schematic.png)|
|:--:|
|**Figure 12:** Circuit diagram of a basic non-inverting integrator|

In [29]:
clearpage()

<IPython.core.display.Latex object>

## Symbolic analysis
Assuming that the amplifiers are ideal, we can pretend as though the current through the input resistor is "flowing" directly to ground -- since ideal amplifier will always "attempt" to eliminate the difference between its two inputs. With that in mind, the time-domain analysis becomes much easier:
$$
    -v_\mathrm{in} + iR = 0 
$$
By Kircchoff's current law, the current through the resistor is equal to the current through the feedback capacitor:
$$
    i = \frac{\mathrm{d}v_C}{\mathrm{d}t}
$$
Since the current through the capacitor can only flow from the junction to the amplifer output, $v_c = - v_\mathrm{out}$, meaning that
$$
    -v_\mathrm{in} - RC\frac{\mathrm{d}v_\mathrm{mid}}{\mathrm{d}t} = 0
$$
or equivalently,
$$
    \frac{\mathrm{d}v_\mathrm{mid}}{\mathrm{d}t} = -\frac{1}{RC}v_\mathrm{in}
$$
Integrating on both sides yields
$$
    v_\mathrm{mid} - v_0 = -\frac{1}{RC}\int_0^t{v_\mathrm{in}(t')\,\mathrm{d}t'}
$$
and with $v_0 = 0$, we get
$$
    v_\mathrm{mid} = -\frac{1}{RC}\int_0^t{v_\mathrm{in}(t')\,\mathrm{d}t'}
$$
Since we have an inverting amplifier in the second stage, we get
$$
    v_\mathrm{out} = \frac{A}{RC}\int_0^t{v_\mathrm{in}(t')\,\mathrm{d}t'}
$$
where $A = R_f / R_i$ is the amplifier gain.

### Example with sinusoidal input
From basic calculus, we recall that
$$
    \int_0^t{\sin(\omega t')\,\mathrm{d}t'} = \frac{1-\cos(\omega t)}{\omega}
$$
and with $v_\mathrm{in} = \hat{v}\sin(\omega t')$, we can reasonably expect the output voltage to be $v_\mathrm{out} = k\left[1-\cos(\omega t)\right]$, with $k = A/(\omega RC)$ being our scaling factor.

We can see in the diagram for the middle circuit that $A = 628.3\,\mathrm{k}\Omega / 10.0\,\mathrm{k}\Omega = 62.83$, which is close to the true value of $2\pi \cdot 10$. For the sake of clarity, we will use $A = 2\pi\cdot 10$. Henceforth
$$
    k = \frac{2\pi}{\omega} \cdot \frac{10}{RC} = \frac{10}{fRC}
$$
where $f$ is the *temporal* frequency of the input signal. With $\hat{v} = 1.0\,\mathrm{V}$, $f = 1000\,\mathrm{Hz}$, $R = 10.0\,\mathrm{k}\Omega$ and $C = 1.0\,\mu\mathrm{F}$, the output voltage magnitude will be
$$
    \hat{v}_\mathrm{out} = k\hat{v} = 1.0\,\mathrm{V}\cdot\frac{10}{1000\,\mathrm{Hz} \cdot 10.0\,\mathrm{k}\Omega \cdot 1.0\,\mu\mathrm{F}} \approxeq 1.0\,\mathrm{V}
$$
so that $v_\mathrm{min} = 0.0\,\mathrm{V}$ and $v_\mathrm{max} = 2.0\,\mathrm{V}$.

## Results

In [30]:
disp_imgfig("result_task_4-1.png", 
            "Transient analysis of two integrators (blue and orange trace) \
            and a reference signal (yellow trace)")

|![Transient analysis of two integrators (blue and orange trace)             and a reference signal (yellow trace)](result_task_4-1.png)|
|:--:|
|**Figure 13:** Transient analysis of two integrators (blue and orange trace)             and a reference signal (yellow trace)|

## Discussion
The simulated results aren't particularly suprising or interesting, since we have already covered most of the nuances in the symbolic analysis part of this section. With that said, the output voltage magnitude of the first integrator (see figure 13; blue trace) is neither an integer multiple, nor a rational multiple, of the input voltage magnitude. The output voltage magnitude of the second integrator (see figure 13; orange trace) *is* an integer multiple of the input voltage magnitude -- as discussed in the symbolic analysis part.

In [31]:
clearpage()

<IPython.core.display.Latex object>

# Wien bridge filter with input
## Schematics

In [32]:
disp_imgfig("lab-b-task-5_1-schematic.png", "Circuit diagram of a Wien bridge filter")

|![Circuit diagram of a Wien bridge filter](lab-b-task-5_1-schematic.png)|
|:--:|
|**Figure 14:** Circuit diagram of a Wien bridge filter|

## Symbolic analysis
We only need to analyze the denominator of the transfer function, which describes the poles of the system. The characteristic equation of the system is given by
$$
    \chi(s) = s^2 + \frac{3-G}{RC}s + \left(\frac{1}{RC}\right)^2
$$
which lets us quickly identify the natural frequency $\omega_0$ and the damping factor $\zeta$, since the characteristic equation of a generic second-order system is
$$
    \chi(s) = s^2 + 2\zeta\omega_0 s + \omega_0^2
$$
With that in mind, we have
\begin{align*}
    \omega_0 = \frac{1}{RC} && \zeta = \frac{3-G}{2}
\end{align*}
and with $f_0 = \omega_0 / 2\pi$ and $Q = 1/(2\zeta)$, we get
\begin{align*}
    f_0 = \frac{1}{2\pi RC} && Q = \frac{1}{2(3-G)/2} = \frac{1}{3-G}
\end{align*}

### Numerical example
Using $R = 10.0\,\mathrm{k}\Omega$ and $C = 10.0\,\mathrm{nF}$, we get
\begin{align*}
    \omega_0 &= \frac{1}{10.0\,\mathrm{k}\Omega \cdot 10.0\,\mathrm{nF}} \approx 10.0\,\mathrm{kHz} \\
    f_0 &= \frac{1}{2\pi \cdot 10.0\,\mathrm{k}\Omega \cdot 10.0\,\mathrm{nF}} \approx 1.59\,\mathrm{kHz} \\
    \zeta(G = 2) &= \frac{3 - 2}{2} = 0.50 \\
    Q(G = 2) &= \frac{1}{3 - 2} = 1.00
\end{align*}

## Results
### AC analysis of the original circuit

In [33]:
disp_imgfig("result_task_5-1.png", "A Bode plot of the original circuit")

|![A Bode plot of the original circuit](result_task_5-1.png)|
|:--:|
|**Figure 15:** A Bode plot of the original circuit|

### Transient analysis of the altered circuit

In [34]:
disp_imgfig("result_task_5-1-b.png",
            "Transient analysis of the altered circuit in time domain " + \
            r"($R_{f_2} = 32\,\mathrm{k}\Omega$, " + \
            r"run time $T = 1.000\,\mathrm{s}$, " + \
            r"time step $\Delta t = 10\,\mu\mathrm{s}$, " + \
            r"truncated to $t = 10.0\,\mathrm{ms}$)")

|![Transient analysis of the altered circuit in time domain ($R_{f_2} = 32\,\mathrm{k}\Omega$, run time $T = 1.000\,\mathrm{s}$, time step $\Delta t = 10\,\mu\mathrm{s}$, truncated to $t = 10.0\,\mathrm{ms}$)](result_task_5-1-b.png)|
|:--:|
|**Figure 16:** Transient analysis of the altered circuit in time domain ($R_{f_2} = 32\,\mathrm{k}\Omega$, run time $T = 1.000\,\mathrm{s}$, time step $\Delta t = 10\,\mu\mathrm{s}$, truncated to $t = 10.0\,\mathrm{ms}$)|

In [35]:
%%capture _
file = open("result_task_5-1_transient.csv")   # open file

t = []   # time
x = []   # output value

for l in file.readlines():
    r = l.strip().split(",")
    
    try:
        T = float(eval(r[0])) 
        value = float(eval(r[1]))
    except:
        continue
    t.append(T)
    x.append(value)

# Clean up the namespace
del T

t_a = np.array(t)
x_a = np.array(x)

# Perform normalized FFT
bins, X, peaks = norm_fft(t_a, x_a, thr=0.25)
X_mag = np.abs(X)

freq, coeff = bins[peaks], X_mag[peaks]

# Plot the figure
fig = plt.figure()
plt.plot(bins, X_mag)
plt.plot(freq, coeff, "o")
plt.xlim(5.0e2)

for xy in zip(freq, coeff):
    plt.annotate('(%.0f, %.3f)' % xy, xy=xy, textcoords='data')

plt.xlabel("Frequency [Hz]")
plt.ylabel("Coefficient [V]")

plt.xscale("log")
plt.grid()

In [36]:
disp_pltfig(fig, "Normalized FFT of the transient analysis values")

|![Normalized FFT of the transient analysis values](tmpfigs/tmpfig_17.png)|
|:--:|
|**Figure 17:** Normalized FFT of the transient analysis values|

### Effects of a pulse signal at the input

In [37]:
disp_imgfig("result_task_5-2.png",
            "Transient analysis of the altered circuit with a pulse signal at the input" + \
            r"($R_{f_2} = 35\,\mathrm{k}\Omega$, " + \
            r"run time $T = 1.000,\mathrm{s}$, " + \
            r"time step $\Delta t = 10\,\mu\mathrm{s}$, " + \
            r"truncated to $t = 10.0\,\mathrm{ms}$)")

|![Transient analysis of the altered circuit with a pulse signal at the input($R_{f_2} = 35\,\mathrm{k}\Omega$, run time $T = 1.000,\mathrm{s}$, time step $\Delta t = 10\,\mu\mathrm{s}$, truncated to $t = 10.0\,\mathrm{ms}$)](result_task_5-2.png)|
|:--:|
|**Figure 18:** Transient analysis of the altered circuit with a pulse signal at the input($R_{f_2} = 35\,\mathrm{k}\Omega$, run time $T = 1.000,\mathrm{s}$, time step $\Delta t = 10\,\mu\mathrm{s}$, truncated to $t = 10.0\,\mathrm{ms}$)|

In [38]:
%%capture _
file = open("result_task_5-2_transient.csv")   # open file

t = []   # time
x = []   # output value

for l in file.readlines():
    r = l.strip().split(",")
    
    try:
        T = float(eval(r[0])) 
        value = float(eval(r[1]))
    except:
        continue
    t.append(T)
    x.append(value)

# Clean up the namespace
del T

t_a = np.array(t)
x_a = np.array(x)

# Perform normalized FFT
bins, X, peaks = norm_fft(t_a, x_a, thr=0.25)
X_mag = np.abs(X)

freq, coeff = bins[peaks], X_mag[peaks]

# Plot the figure
fig = plt.figure()
plt.plot(bins, X_mag)
plt.plot(freq, coeff, "o")
plt.xlim(5.0e2)

for xy in zip(freq, coeff):
    plt.annotate('(%.0f, %.3f)' % xy, xy=xy, textcoords='data')

plt.xlabel("Frequency [Hz]")
plt.ylabel("Coefficient [V]")

plt.xscale("log")
plt.grid()

In [39]:
disp_pltfig(fig, "Normalized FFT of the transient analysis values")

|![Normalized FFT of the transient analysis values](tmpfigs/tmpfig_19.png)|
|:--:|
|**Figure 19:** Normalized FFT of the transient analysis values|

## Discussion
### Altering the original circuit
If we were to alter the original circuit, so that we get a high Q factor, then we can expect strong resonance from the circuit. What this practically means, is that the Bode plot becomes "sharper" around the resonant frequency. Additionally, the "steepness" of the phase diagram becomes more severe as the Q factor increases.

### Describing the altered circuit
It isn't immediately obvious what is going on under the hood if we were to just look at the voltage at the output with respect to time (see figure 16), though there are clues that we can pick up on without having to look at the frequency components of the signal. From the very beginning, the input signal dominates as the system does not have enough internal energy to "overpower" the input signal -- but as the system starts resonating, it starts building up internal energy until it ultimately overpowers the input signal. We finally have a steady state, where the system's resonant frequency dominates.

We can also observe this behavior when we break down the signal into its frequency components (see figure 17). The three peaks shown in the figure are the ones with a coefficient of at least $0.25\,\mathrm{V}$ -- this threshold was chosen so that the plot wouldn't get too crowded with insignificant peaks. The most obvious peak is the one at the system's resonant frequency at $f = 1556\,\mathrm{Hz}$, which the system settles to as time progresses. The other peak is at $f = 3113\,\mathrm{Hz}$, which is roughly double the resonant frequency. The final peak is at $f = 10000\,\mathrm{Hz}$, which is the frequency of our injected signal.

### Effects of a pulse signal at the input
We can see in the time domain result (???) that the figure starts oscillating on its own as soon as a voltage is provided at the input. Even as the voltage is removed from the input -- as is the case with a brief pulse -- the oscillation is self-sustaining and remarkably stable. We can see in the frequency decomposition of the transient analysis values (see figure ???) that the two strongest peaks are at the system's resonant frequency and the system's second harmonic respectively. The other peaks are probably caused by the harmonics contained in the initial square pulse (see figure ???).

# Wien bridge without input
## Schematics

In [40]:
disp_imgfig("lab-b-task-5_3-schematic.png", "Circuit diagram of a test Wien bridge oscillator")

|![Circuit diagram of a test Wien bridge oscillator](lab-b-task-5_3-schematic.png)|
|:--:|
|**Figure 20:** Circuit diagram of a test Wien bridge oscillator|

In [43]:
clearpage()

<IPython.core.display.Latex object>

## Results

In [41]:
disp_imgfig("result_task_5-3.png", "Transient analysis of a Wien bridge oscillator")

|![Transient analysis of a Wien bridge oscillator](result_task_5-3.png)|
|:--:|
|**Figure 21:** Transient analysis of a Wien bridge oscillator|

## Discussion

The provided circuit diagram does *not* depict a self-starting oscillator, as it is in the inverting configuration -- which does not satisfy the Barkhausen conditions. The circuit diagram that is depicted in this section *does* start, but only if an initial input signal is provided.

In practice, thermal noise and internal imperfections are sufficient to start the oscillation, so long as the loop gain is sufficiently high. This ultimately means that the resistances $R_f$ and $R_g$ must be carefully chosen in order for the loop gain to be greater than or equal to 1. For most practical applications, $R_f$ and $R_g$ are chosen so that $R_f / R_g \geq 2$.

# Administration - rendering
If you are seeing this section, then this notebook has been improperly rendered - and will be littered with code and other cells that are supposed to be hidden from general readers.

The code below renders this notebook into a PDF file. As the notebook itself has a compliant name, the PDF file will also be rendered with a compliant name.

In [45]:
# The question is: can a cell hide itself
print(f" I: Exporting notebook as PDF... (output file: {pdfname})", file=sys.stderr)
print(f" I: Current path: {os.path.realpath('.')}", file=sys.stderr)
_ = os.system(f'\
jupyter nbconvert \
--to pdf \
--TagRemovePreprocessor.remove_cell_tags="hide-cell" \
--TagRemovePreprocessor.remove_input_tags="hide-input" \
{nbname}')

# --- Nothing that is hidden, can be shown beyond this point --- #

 I: Exporting notebook as PDF... (output file: FYS3220_LAB_B_V25_jicalder.pdf)
 I: Current path: /home/johnc/Documents/Notebooks/FYS3220_LAB_B_V25_jicalder
[NbConvertApp] Converting notebook FYS3220_LAB_B_V25_jicalder.ipynb to pdf
[NbConvertApp] Writing 54752 bytes to notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', 'notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: ['bibtex', 'notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 1131095 bytes to FYS3220_LAB_B_V25_jicalder.pdf
